In [1]:
import pandas as pd 
import pickle

In [2]:
pd.options.display.float_format = '{:,.2f}'.format

In [3]:
# make a dict of team ids for future reference

teams_df = pd.read_csv('../data/Teams.csv')
team_dict = dict(zip(teams_df["Team_Id"], teams_df["Team_Name"]))

In [4]:
# read in the data and combine the regular season and tournament data

reg_season_df = pd.read_csv('../data/RegularSeasonDetailedResults.csv')
tourney_df = pd.read_csv('../data/TourneyDetailedResults.csv')

allresults_df = pd.concat((reg_season_df, tourney_df), ignore_index=True)

In [5]:
# 2017 data is uploaded at a later stage. Once we have it:

reg_season2017_df = pd.read_csv('../data/2017_Final_DetailedResults.csv')
combined_df = pd.concat((allresults_df, reg_season2017_df), ignore_index=True)

In [6]:
with open("../pickled_data/team_dict.pickle", "wb") as td_file:
    pickle.dump(team_dict, td_file)
td_file.close()

In [7]:
print len(tourney_df)
print len(tourney_df[tourney_df["Season"] > 2007])
print len(combined_df[combined_df["Season"] > 2007])

914
594
53568


In [8]:
combined_df["Wteamname"] = combined_df["Wteam"].map(lambda x: team_dict[x])
combined_df["Lteamname"] = combined_df["Lteam"].map(lambda x: team_dict[x])
combined_df.sort_values(by=["Season", "Daynum"], inplace=True)
combined_df.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,Wfga,...,Lfta,Lor,Ldr,Last,Lto,Lstl,Lblk,Lpf,Wteamname,Lteamname
0,2003,10,1104,68,1328,62,N,0,27,58,...,22,10,22,8,18,9,2,20,Alabama,Oklahoma
1,2003,10,1272,70,1393,63,N,0,26,62,...,20,20,25,7,12,8,6,16,Memphis,Syracuse
2,2003,11,1266,73,1437,61,N,0,24,58,...,23,31,22,9,12,2,5,23,Marquette,Villanova
3,2003,11,1296,56,1457,50,N,0,18,38,...,15,17,20,9,19,4,3,23,N Illinois,Winthrop
4,2003,11,1400,77,1208,71,N,0,30,61,...,27,21,15,12,10,7,1,14,Texas,Georgia


In [9]:
def get_pct(df, iterable):
	for tup in iterable:
		name = tup[0] + "%"
		df[name] = ((df[tup[0]]/df[tup[1]]) * 100)

	cols_to_drop = [item for tup in iterable for item in tup]
	df.drop(cols_to_drop, axis=1, inplace=True)
	return df 

In [10]:
pct_cols = [("Wfgm", "Wfga"), ("Wfgm3", "Wfga3"), ("Wftm", "Wfta"), ("Lfgm", "Lfga"), ("Lfgm3", "Lfga3"), ("Lftm", "Lfta"),]
combined_df = get_pct(combined_df, pct_cols)

In [11]:
combined_df["W_+/-"] = combined_df["Wto"] - combined_df["Wstl"]
combined_df["L_+/-"] = combined_df["Lto"] - combined_df["Lstl"]

In [12]:
more_cols_todrop = ["Numot", "Wpf", "Lpf", "Wloc"]
combined_df.drop(more_cols_todrop, axis=1, inplace=True)

combined_df.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wor,Wdr,Wast,Wto,...,Wteamname,Lteamname,Wfgm%,Wfgm3%,Wftm%,Lfgm%,Lfgm3%,Lftm%,W_+/-,L_+/-
0,2003,10,1104,68,1328,62,14,24,13,23,...,Alabama,Oklahoma,46.55,21.43,61.11,41.51,20.00,72.73,16,9
1,2003,10,1272,70,1393,63,15,28,16,13,...,Memphis,Syracuse,41.94,40.00,52.63,35.82,25.00,45.00,9,4
2,2003,11,1266,73,1437,61,17,26,15,10,...,Marquette,Villanova,41.38,44.44,58.62,30.14,11.54,60.87,5,10
3,2003,11,1296,56,1457,50,6,19,11,12,...,N Illinois,Winthrop,47.37,33.33,54.84,36.73,27.27,53.33,-2,15
4,2003,11,1400,77,1208,71,17,22,12,14,...,Texas,Georgia,49.18,42.86,84.62,38.71,37.50,62.96,10,3


In [13]:
ncaa_df = combined_df[combined_df["Season"] > 2007]
print len(ncaa_df)
ncaa_df.tail()

53568


,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wor,Wdr,Wast,Wto,...,Wteamname,Lteamname,Wfgm%,Wfgm3%,Wftm%,Lfgm%,Lfgm3%,Lftm%,W_+/-,L_+/-
77545,2017,132,1276,71,1458,56,3,22,17,9,...,Michigan,Wisconsin,56.25,43.48,77.78,39.29,40.00,75.00,2,11
77546,2017,132,1343,71,1463,59,9,23,15,7,...,Princeton,Yale,48.08,42.31,71.43,37.74,30.00,68.42,0,3
77547,2017,132,1348,70,1433,63,9,27,12,6,...,Rhode Island,VA Commonwealth,44.44,40.00,73.68,31.34,28.57,77.27,3,1
77548,2017,132,1374,71,1153,56,7,27,14,8,...,SMU,Cincinnati,50.00,52.63,69.23,31.15,16.67,77.78,6,0
77549,2017,132,1407,59,1402,53,14,19,5,5,...,Troy,Texas St,35.00,5.88,84.21,41.67,35.29,87.50,-5,16


In [14]:
ncaa_df.to_csv('../proc_data/CombinedResults.csv', header=True)